<a href="https://colab.research.google.com/github/jekrami/Astro/blob/main/AI_MultiAgent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!curl https://ollama.com/install.sh | sh
!pip install langchain
!pip install -qU langchain-openai
!pip install transformers
!pip install IPython
!pip install web-browser
!pip install numpy
!pip install pandas
!pip install torch
!pip install youtube-search-python
!pip install youtube-transcript-api
!pip install faiss-cpu
!pip install ollama
!pip install 'crewai[tools]'
!pip install Markdown

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.4/404.4 kB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.4 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.7/

Load Libraries

In [ ]:
# General
import re
import os
import webbrowser
import numpy as np
import pandas as pd
from IPython.display import display, HTML, Audio
# DL
import torch
# Youtube
from youtubesearchpython import VideosSearch
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled, NoTranscriptFound
# Huggingface
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoModelForSeq2SeqLM
from transformers import DPRQuestionEncoder, DPRContextEncoder, DPRQuestionEncoderTokenizer, DPRContextEncoderTokenizer
# Vector Databases
import faiss
# Langchin | Elevenlabs | Langchain Agents
from langchain import LLMChain
from langchain.prompts import PromptTemplate
from langchain.agents import AgentType, initialize_agent
##, load_tools
from langchain_community.tools.eleven_labs.text2speech import ElevenLabsText2SpeechTool
# Ollama
import ollama
from markdown import Markdown


In [ ]:
# Constants (predefined )
DESTINATION = "Amsterdam"
PREFRENCES = ["Museums","Outdorr Activities"]
MAX_RESULTS = 20
MIN_VIEWS = 10000
LLM = "facebook/bart-large-cnn"
LOCAL_LLM = "llama3"
MAX_TOKENS = 1000

In [ ]:
def parse_views(views_str):
  """
  Parses the views string from YouTube and converts it to an integer.
  5,909 views -> 5909
  """
# Remove the "views" part and any commas
  views_str = views_str.lower().replace('views', '').replace(',', '').strip()

  return int(views_str) # In case of any parsing error

def fetch_youtube_videos (destination, preferences, MIN_VIEWS, max_results=10):
  """
  Fetches relevant YouTube videos based on the destination and user preferences. I
  Parameters:
  - destination (str): The travel destination (e.g., 'Amsterdam').
  - preferences (list): List of user-selected preferences (e.g., ['Museums', 'Outdoor Activities']). - max_results (int): Maximum number of videos to fetch.
  Returns:
  - videos (list): List of dictionaries containing video details with views parsed as integers.
  # Combine preferences into a search query
  """
  preferences_query= ' '.join(preferences)
  search_query = f"{destination} travel guide {preferences_query} Netherlands"
  # Initialize VideosSearch
  videos_search= VideosSearch(search_query, limit=max_results)
  # Execute search
  search_results = videos_search.result()

  videos = []
  for video in search_results['result']:
    views_str = video['viewCount']['text']
    views = parse_views (views_str)
    print (f"parseview {views}")

    # Filter out videos with fewer than MIN_VIEWS
    if views < MIN_VIEWS:
      continue # Skip this video

    video_data = {
      'Title': video['title'],
      'Duration': video['duration'],
      'Views': views, # Store as integer
      'Channel': video['channel']['name'],
      'Link': video['link']
    }
    videos.append(video_data)
  return videos


In [ ]:
videos = fetch_youtube_videos(DESTINATION, PREFRENCES, MIN_VIEWS, MAX_RESULTS)

parseview 204005
parseview 284331
parseview 92073
parseview 6279
parseview 1070679
parseview 1019734
parseview 5423
parseview 676116
parseview 174281
parseview 376986
parseview 238783
parseview 107422
parseview 40024
parseview 230974
parseview 5411
parseview 2267900
parseview 3069214
parseview 178786
parseview 18164
parseview 14454


In [ ]:
len(videos)

17

In [ ]:
videos

[{'Title': 'BEST THINGS TO DO IN AMSTERDAM FOR FIRST TIMERS W/ MAP (2024) | 20+ Must-Dos, Hidden Gems & More!',
  'Duration': '19:39',
  'Views': 204005,
  'Channel': 'Happy to Wander',
  'Link': 'https://www.youtube.com/watch?v=cqYvuRXwaVA'},
 {'Title': 'Top 10 Places to Visit in Amsterdam 2024 | Netherlands Travel Guide',
  'Duration': '12:01',
  'Views': 284331,
  'Channel': 'TRIP XTREME',
  'Link': 'https://www.youtube.com/watch?v=3izVLop9iKg'},
 {'Title': 'Top 7 Best Things To Do in Amsterdam (2024) | Ultimate Travel Guide',
  'Duration': '8:45',
  'Views': 92073,
  'Channel': 'Travel Awaits You',
  'Link': 'https://www.youtube.com/watch?v=QEAQ8N6SK6w'},
 {'Title': 'The BEST things to do in Amsterdam 🇳🇱🍻 handpicked by the locals. #Amsterdam #cityguide',
  'Duration': '6:17',
  'Views': 1070679,
  'Channel': 'Withlocals',
  'Link': 'https://www.youtube.com/watch?v=GRrXVAFh_XU'},
 {'Title': '10 Amazing Places to Visit in the Netherlands 4K  🇳🇱  | Netherlands Travel Guide',
  'Durati

In [ ]:
def display_videos(videos):
  """
  Displays the list of videos in a pandas DataFrame and optionally opens them in the browser.
  Parameters:
  - videos (list): List of dictionaries containing video details.
  """
  if not videos:
    print("No videos found with more than 10,000 views.")
    return
  # Create a DataFrame for better display
  videos_df = pd.DataFrame(videos)
  print("\nFetched YouTube Videos (Filtered by >10,000 views):")
  print(videos_df [['Title', 'Duration', 'Views', 'Channel']].to_string(index=False))

  # Optionally, ask the user if they want to open the videos in the browser
##  open_browser = input("\nDo you want to open these videos in your browser? (y/n): ").strip().lower()
##  if open_browser == 'y':
##    for video in videos:
##    webbrowser.open(video [ 'Link'])
##  display_videos (videos)


In [ ]:
# Display clickable links
print("\n### Watch These Videos: ")
for idx, video in enumerate(videos, 1):
  # Display as markdown link
  display (HTML (f"{idx}. <a href='{video['Link']}' target='_blank'>{video['Title']}</a>"))



### Watch These Videos: 


In [ ]:
display_videos(videos)


Fetched YouTube Videos (Filtered by >10,000 views):
                                                                                              Title Duration   Views                Channel
  BEST THINGS TO DO IN AMSTERDAM FOR FIRST TIMERS W/ MAP (2024) | 20+ Must-Dos, Hidden Gems & More!    19:39  204005        Happy to Wander
                                Top 10 Places to Visit in Amsterdam 2024 | Netherlands Travel Guide    12:01  284331            TRIP XTREME
                                Top 7 Best Things To Do in Amsterdam (2024) | Ultimate Travel Guide     8:45   92073      Travel Awaits You
             The BEST things to do in Amsterdam 🇳🇱🍻 handpicked by the locals. #Amsterdam #cityguide     6:17 1070679             Withlocals
                   10 Amazing Places to Visit in the Netherlands 4K  🇳🇱  | Netherlands Travel Guide    11:48 1019734          Lifestyle Hal
                                   TOP 20 Things to Do in AMSTERDAM Netherlands 2024 | Travel Guide    18:1

In [ ]:
def extract_video_id(youtube_url):
  """
  Extracts the video ID from a YouTube URL.
  Parameters:
  - youtube_url (str): The full YouTube video URL.
  Returns:
  - video_id (str): The extracted video ID.
  """
  # Regular expression to extract video ID
  video_id_match = re.search(r'(?:v=|\/)([0-9A-Za-z_-]{11}).*', youtube_url)
  if video_id_match:
    return video_id_match.group(1)
  else:
    return None


In [ ]:
def extract_transcripts(videos_df):
  # Initialize a new column for transcripts
  videos_df[ 'Transcript'] = None

  # Iterate over each video and fetch the transcript
  for index, row in videos_df.iterrows():
    youtube_url = row['Link']
    video_title = row['Title']
    video_id = extract_video_id (youtube_url)

    if video_id:
      try:
        # Fetch the transcript using the video ID
        transcript_list = YouTubeTranscriptApi.get_transcript(video_id)

        # Combine the transcript segments into a single string
        transcript = ' '.join([segment['text'] for segment in transcript_list])

        # Assign the transcript to the DataFrame
        videos_df.at[index, 'Transcript'] = transcript
        print(f"Transcript fetched for video: '{video_title}'")

      except TranscriptsDisabled:
        print(f"Transcripts are disabled for video: '{video_title}'.")
        videos_df.at[index, 'Transcript']
      except NoTranscriptFound:
        print(f"No transcripts found for video: '{video_title}'.")
        videos_df.at[index, 'Transcript'] = "No transcripts found for this video."
      except Exception as e:
        print(f"An error occurred while fetching transcript for video: '{video_title}'. Error: {e}")
        videos_df.at[index, 'Transcript'] = f"Error fetching transcript: {e}"
    else:
      print(f"Could not extract video ID from URL: {youtube_url}")
      videos_df.at[index, 'Transcript'] = "No transcripts found for this video."
  return videos_df



In [ ]:
videos_df = pd.DataFrame(videos)
videos_df = extract_transcripts(videos_df)

Transcript fetched for video: 'BEST THINGS TO DO IN AMSTERDAM FOR FIRST TIMERS W/ MAP (2024) | 20+ Must-Dos, Hidden Gems & More!'
Transcript fetched for video: 'Top 10 Places to Visit in Amsterdam 2024 | Netherlands Travel Guide'
Transcript fetched for video: 'Top 7 Best Things To Do in Amsterdam (2024) | Ultimate Travel Guide'
Transcript fetched for video: 'The BEST things to do in Amsterdam 🇳🇱🍻 handpicked by the locals. #Amsterdam #cityguide'
Transcript fetched for video: '10 Amazing Places to Visit in the Netherlands 4K  🇳🇱  | Netherlands Travel Guide'
Transcript fetched for video: 'TOP 20 Things to Do in AMSTERDAM Netherlands 2024 | Travel Guide'
Transcript fetched for video: 'AMSTERDAM TRAVEL TIPS FOR FIRST TIMERS | 30+ Must-Knows Before Visiting Amsterdam + What NOT to Do!'
Transcript fetched for video: 'Things To Do In Amsterdam: 3 Day Travel Guide'
Transcript fetched for video: 'Visit Amsterdam - Free Things to Do in Amsterdam, The Netherlands'
Transcript fetched for video: 'Ho

In [ ]:
# Initialize the tokenizer
tokenizer  = AutoTokenizer.from_pretrained (LLM)
summarizer = pipeline("summarization", model=LLM)

# Clean text function
def clean_text(text):
  text = re.sub(r'[^\x00-\x7F]+', ' ', text)
  text = re.sub(r'\n+', ' ', text)
  text = re.sub(r'\s+', ' ', text).strip()
  return text

def split_text_into_chunks (text, max_tokens=MAX_TOKENS):
  tokens = tokenizer.encode(text)
  chunks = [tokens[i:i + max_tokens] for i in range(0, len(tokens), max_tokens)]
  return [tokenizer.decode(chunk) for chunk in chunks]

# Split the text into chunks
t = clean_text(videos_df['Transcript'][0])
chunks = split_text_into_chunks(t)

# Summarize each chunk
summaries = [summarizer(chunk, max_length=130, min_length=30, do_sample=False)[0]['summary_text'] for chunk in chunks]

# Combine the summaries if needed
final_summary = ' '.join(summaries)

final_summary


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is 

"If there's one thing you need to try in Amsterdam, make sure it's stroopwafels. The Red Light District is an infamous and oft-misunderstood part of Amsterdam. If you're looking for the best most Amsterdammy thing to do during your visit, it's time to hurl yourself on a boat. The Rijksmuseum is to Amsterdam as the Louvre is to Paris, it's an astounding collection of 8,000 plus pieces. Straat is one of the coolest museums in Amsterdam hands down, located across the river and about a 10-minute free ferry ride from Amsterdam Central Station. One of the best things to do in Amsterdam is just stare at everything and drool. Just outside of Amsterdam Centraal Station is an avenue known as Damrak where you can get a shot of the famous dancing houses. The Skinny Bridge was featured in the 1971 James Bond film Diamonds Are Forever and at night it's lit up by tons of little light bulbs. Amsterdam's canal houses are one of the city's signature elements. The Jewish Cultural Quarter is dedicated to 

In [ ]:
query_encoder = DPRQuestionEncoder.from_pretrained('facebook/dpr-question_encoder-single-nq-base')
query_tokenizer = DPRQuestionEncoderTokenizer.from_pretrained('facebook/dpr-question_encoder-single-nq-base')

passage_encoder = DPRContextEncoder.from_pretrained('facebook/dpr-ctx_encoder-single-nq-base')
passage_tokenizer = DPRContextEncoderTokenizer.from_pretrained('facebook/dpr-ctx_encoder-single-nq-base')


Some weights of the model checkpoint at facebook/dpr-question_encoder-single-nq-base were not used when initializing DPRQuestionEncoder: ['question_encoder.bert_model.pooler.dense.bias', 'question_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRQuestionEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRQuestionEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at facebook/dpr-ctx_encoder-single-nq-base were not used when initializing DPRContextEncoder: ['ctx_encoder.bert_model.pooler.dense.bias', 'ctx_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRContextEncoder from the


4.2 Encode Passages (Transcripts)
We will encode the passages (i.e., parts of the transcript) and store their embeddings for efficient retrieval.

In [ ]:
# Encode passages (transcripts) using the DPR context encoder
def encode_passages(videos_df):
  passages = videos_df['Transcript'].tolist()
  passage_embeddings = []

  for passage in passages:
    inputs = passage_tokenizer(passage, return_tensors='pt', max_length=512, truncation=True, padding=True)
    with torch.no_grad():
      embedding = passage_encoder(**inputs). pooler_output
    passage_embeddings.append(embedding.numpy())

  # Convert to a numpy array
  passage_embeddings =  np.vstack(passage_embeddings)

  return passage_embeddings

# Encode all the transcripts in the DataFrame
passage_embeddings = encode_passages(videos_df)


In [ ]:
passage_embeddings


array([[ 0.2508697 , -0.08455344,  0.2526211 , ..., -0.515923  ,
         0.35670117, -0.35892242],
       [ 0.16322616, -0.14555949,  0.04292687, ..., -0.46101978,
         0.5941632 , -0.42735475],
       [ 0.22854544, -0.24929877,  0.2448014 , ..., -0.3879951 ,
         0.3089133 , -0.63366103],
       ...,
       [ 0.6625286 ,  0.336885  ,  0.6936485 , ..., -0.6248708 ,
        -0.0705755 , -0.585556  ],
       [ 0.32964692, -0.19305477,  0.12044439, ..., -0.4373814 ,
         0.36157912, -0.23962325],
       [ 0.56125927,  0.09014735,  0.20071362, ..., -0.394173  ,
         0.12358255, -0.95792425]], dtype=float32)

## 4.3 Build a FAISS Index for Fast Retrival

In [ ]:
dimention = passage_embeddings.shape[1]
faiss_index = faiss.IndexFlatIP(dimention)

faiss_index.add(passage_embeddings)


In [ ]:
faiss_index


<faiss.swigfaiss_avx512.IndexFlatIP; proxy of <Swig Object of type 'faiss::IndexFlatIP *' at 0x7bc3fce7a7f0> >

## **4.4** Encode the Query and Retrieve Relevant Passages
Next, we will encode the user's query and use it to retrieve the most relevant passages from the indexed transcripts.

In [ ]:
# Function to encode the query and search for the most relevant passages
def search_relevant_passages(query, faiss_index, top_k=3):
  # Encode the query using the DPR question encoder
  query_inputs = query_tokenizer(query, return_tensors='pt', max_length=128, truncation=True, padding=True)

  with torch.no_grad():
    query_embedding = query_encoder(**query_inputs). pooler_output.numpy()

  # Ensure query_embedding is 2D (i.e., shape (1, embedding_dimension))
  query_embedding = query_embedding.reshape(1, -1)  # Reshaping to (1, embedding_dimension)

  # Search for the top-k most similar passages
  distances, indices = faiss_index.search(query_embedding, top_k)

  print (f"distances: {distances}")
  print (f"indices: {indices}")

  # Retrieve the corresponding transcripts and titles
  retrieved_passages = []
  for i in range(top_k): # Loop through top_k results
    idx = indices[0][i] # Get the index of the passage
    retrieved_passages.append({
      'Title': videos_df.iloc[idx]['Title'], # Get the title using the index
      'Transcript': videos_df.iloc[idx]['Transcript'],  # Get the transcript using the index
      'Similarity Score': distances[0][i] # Get the corresponding similarity score using i, not idx
    })
  return retrieved_passages


#Example query
query= "What are the best museums in Amsterdam?"

# Search for the most relevant passages
retrieved_passages = search_relevant_passages (query, faiss_index, top_k=2)

# Display retrieved results
for passage in retrieved_passages:
  print(f"Title: {passage['Title']}")
  print(f"Transcript: {passage ['Transcript']}")
  print(f"Similarity Score: {passage ['Similarity Score']}\n")



distances: [[79.00554 77.33417]]
indices: [[1 2]]
Title: Top 10 Places to Visit in Amsterdam 2024 | Netherlands Travel Guide
Transcript: Amsterdam the capital of the Netherlands is one of the most popular tourist destinations in Europe with its universities academies and research institutes along with more than 40 museums numerous theaters and entertainment venues Amsterdam is also the country's leading Cultural Center in this video we're going to be taking a look at the top 10 rated attractions and things to do in Amsterdam and just wait till you see the top three that we're going to be showing in this video something you would never even have thought of so make sure you watch till the end before we begin make sure to subscribe to our YouTube channel for more awesome travel guides and make sure you hit the notification Bell so that you know when we publish a new video so now let's cut to the chase at number 10 visit one of the world's oldest botanical gardens Amsterdam offers a surpri

In [ ]:
# Function to encode the query and search for the most relevant passages
def search_relevant_passages(videos_df, faiss_index, query, top_k=3):
  # Encode the query using the DPR question encoder
  query_inputs = query_tokenizer(query, return_tensors='pt', max_length=128, truncation=True, padding=True)

  with torch.no_grad():
    query_embedding = query_encoder(**query_inputs).pooler_output.numpy()

  # Ensure query_embedding is 2D (i.e., shape (1, embedding_dimension))
  query_embedding = query_embedding.reshape(1, -1) # Reshaping to (1, embedding_dimension)

  # Search for the top-k most similar passages
  distances, indices = faiss_index.search(query_embedding, top_k)

  # Filter the DataFrame based on the retrieved indices
  top_k_indices = indices[0][:top_k] # Get the top_k indices from the FAISS search result
  top_k_videos = videos_df.iloc[top_k_indices].copy() # Use iloc to filter the DataFrame based on these indices and create a copy

  # Add the similarity score as a new column in the DataFrame
  top_k_videos['Similarity Score'] = distances[0][:top_k]

  # Add the query as a new column in the DataFrame
  top_k_videos['Query'] = query

  return top_k_videos


# Example query
query= "What are the best museums in Amsterdam?"
# Search for the most relevant passages and filter videos from videos_df
top_k_videos = search_relevant_passages(videos_df, faiss_index, query, top_k=3)

# Display the top_k filtered videos along with the similarity scores and query
print("Filtered Top-k Videos with Similarity Score and Query:")
top_k_videos[['Title', 'Transcript', 'Similarity Score', 'Query']]


Filtered Top-k Videos with Similarity Score and Query:


,Title,Transcript,Similarity Score,Query
1,Top 10 Places to Visit in Amsterdam 2024 | Net...,Amsterdam the capital of the Netherlands is on...,79.005539,What are the best museums in Amsterdam?
2,Top 7 Best Things To Do in Amsterdam (2024) | ...,welcome to Amsterdam a city where the past and...,77.334167,What are the best museums in Amsterdam?
15,Amsterdam Travel Guide 2023 | 10 BEST Tips & T...,Amsterdam the capital city of the Netherlands ...,76.106430,What are the best museums in Amsterdam?


In [ ]:
!ollama serve > rocama.log 2>$1 &

## 5. Building Travel Agent to Generate Top 10 Queries

In [ ]:
# Function to generate questions using LLAMA 3 via Ollama with a refined prompt
def generate_questions(city):
  # Refined prompt to ask specifically for only the questions, without extra text
  prompt = f"""
  As a travel guide expert, generate a list of the top 10 questions that a first-time traveler might ask about visiting {city}.
  Please provide only the questions, numbered 1 to 10, without any additional descriptions.
  """

  # Use Ollama's Chat API to generate the questions
  response = ollama.chat(model="llama3", messages=[{"role": "user", "content": prompt}])

  return response['message']['content']


# Function to display the questions beautifully with Markdown
def display_questions_with_markdown(city):
  # Generate the top 10 questions
  questions_text = generate_questions(city)

  # Convert the text into a Markdown-friendly format
  markdown_output = f"### Top 10 Questions for First-Time Travelers to {city}:\n\n{questions_text}"

  # Display the formatted text using Markdown
  ###display(Markdown(markdown_output))
  display(markdown_output)

# Example usage
city = "Amsterdam"
display_questions_with_markdown(city)


ConnectError: [Errno 111] Connection refused

In [ ]:
import crewai
from crewai import Task, Crew, Process, Agent, LLM
import ollama

# Initialize the local LLAMA 3 model via Ollama
#local_llama3 = ollama.chat (model="llama3")
local_llama3 =LLM(model="ollama/llama3", base_url="http://localhost:11434")

# Define the generator agent with a backstory and detailed parameters
generator = Agent(
  role="Travel Content Creator",
  goal="Generate engaging and relevant travel-related questions for a first-time visitor to Amsterdam",
  backstory="""You are an experienced travel content creator who knows how to generate engaging and insightful questions for first time visitors to Amsterdam.
  You focus on understanding travelers' needs and are able to create questions that address practical aspects of travel, like transportation, attractions, and culture.""",
  verbose=True,
  allow_delegation=False, # In this case, the agent should not delegate the task
  llm=local_llama3 # Use LLAMA 3 via Ollama as the LLM for this agent
)

# Define the summarizer agent for answering the travel questions
summarizer = Agent(
  role="Travel Expert",
  goal="Provide concise and helpful answers to common questions about visiting Amsterdam",
  backstory="""You are an expert in Amsterdam tourism with deep knowledge of the city's transport, attractions, and cultural norms.
  You can provide concise, practical answers to travelers' questions that are easy to understand and apply.""",
  verbose=True,
  allow_delegation=False, # This agent should handle the task independently
  llm=local_llama3 # Use LLAMA 3 for summarizing answers
)

# Define the reviewer agent for reviewing the final output
reviewer = Agent(
  role="Travel Content Reviewer",
  goal="Review the final Markdown document for accuracy, readability, and relevance",
  backstory="""You are a reviewer with extensive experience in travel content. You ensure that the questions and answers are accurate, useful, and the formatting is clean. Your goal is to make sure that the content meets the needs of first-time travelers to Amsterdam.""", verbose=True,
  allow_delegation=False # This agent will handle the review independently
)

# Task to generate top 10 questions for a first-time traveler to Amsterdam
task_generate_questions = Task(
    description="Generate a list of the top 10 questions that a first-time traveler might ask about visiting Amsterdam.",
    agent=generator, # The generator agent is responsible for this task
    expected_output="A list of 10 questions" # Define what output is expected
)

# Task to summarize answers for each question
task_summarize_answers = Task(
    description="Provide a short and concise answer for each of the top 10 questions about visiting Amsterdam.",
    agent=summarizer, # The summarizer agent handles this task
    expected_output="A list of short, concise answers" # Define the expected output for the summary
)

# Task to review the final Markdown output
task_review_output = Task(
    description="Review the final Markdown document for accuracy and readability.",
    agent=reviewer, # The reviewer agent handles this task
    expected_output="Reviewed and accurate Markdown document" # Define the expected reviewed document
)

# Instantiate the Crew of agents with their respective tasks
crew = Crew (
    agents=[generator, summarizer, reviewer], # List of all the agents
    tasks=[task_generate_questions, task_summarize_answers, task_review_output], # The tasks they will perform sequentially
    verbose=True, # Set verbosity level for detailed output logs
    process=Process.sequential, # Process the tasks sequentially
)

# Start the process
result = crew.kickoff()

# Print the final result (the Markdown document)
print(result)





# Agent: Travel Content Creator
## Task: Generate a list of the top 10 questions that a first-time traveler might ask about visiting Amsterdam.


# Agent: Travel Content Creator
## Final Answer: 
Here are the top 10 questions that a first-time traveler might ask about visiting Amsterdam:

1. What's the best way to get around Amsterdam? Are there any specific transportation options you would recommend for getting from the airport to my hotel?
2. Can you tell me more about the city's main attractions and what I should prioritize seeing on my first visit? Should I book tickets in advance or just wing it?
3. What are some must-try foods and drinks while I'm in Amsterdam? Are there any local specialties that I won't want to miss?
4. How do I navigate the city's canals and bridges? Are there any specific routes or landmarks I should check out?
5. What's the deal with the bike lanes in Amsterdam? Are they safe for tourists, and are there any tips you have for navigating them?
6. Can you recom

ERROR:root:LiteLLM call failed: litellm.AuthenticationError: AuthenticationError: OpenAIException - The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable
ERROR:root:LiteLLM call failed: litellm.AuthenticationError: AuthenticationError: OpenAIException - The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable
ERROR:root:LiteLLM call failed: litellm.AuthenticationError: AuthenticationError: OpenAIException - The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable




# Agent: Travel Expert
## Final Answer: 
Here are the top 10 questions that a first-time traveler might ask about visiting Amsterdam, along with short and concise answers:

1. What's the best way to get around Amsterdam? Are there any specific transportation options you would recommend for getting from the airport to my hotel?

Take the train from Schiphol Airport to Centraal Station (approximately €5), then grab a tram or bus to your hotel. Alternatively, book an airport taxi or shuttle service.

2. Can you tell me more about the city's main attractions and what I should prioritize seeing on my first visit? Should I book tickets in advance or just wing it?

Visit the Rijksmuseum, Van Gogh Museum, Anne Frank House, and Vondelpark. Book tickets for popular attractions like the Anne Frank House and Van Gogh Museum in advance to avoid long queues.

3. What are some must-try foods and drinks while I'm in Amsterdam? Are there any local specialties that I won't want to miss?

Try herring, 

AuthenticationError: litellm.AuthenticationError: AuthenticationError: OpenAIException - The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [ ]:
!pip install elevenlabs

from langchain import LLMChain
from langchain.prompts import PromptTemplate
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain_community.tools.eleven_labs.text2speech import ElevenLabsText2SpeechTool

ModuleNotFoundError: No module named 'langchain'

In [ ]:
os.environ["ELEVEN_API_KEY"] = "sk_0fe284ca35492e47e129671eb6137efc73bc17a9bf41a67c"

In [ ]:

text_to_speak = '''Top 10 Questions for First-Time Travelers to Amsterdam:
Here are the top 10 questions that a first-time traveler might ask about visiting Amsterdam:
What is the best way to get around Amsterdam?
Is Amsterdam safe for tourists?
What are some must-see attractions in Amsterdam?
Can I drink the tap water in Amsterdam?
Are there any specific dress code or cultural norms I should be aware of?
How much money do I need to budget for food and activities?
Is Amsterdam a good place for solo travelers or couples?
What are some popular neighborhoods or areas to stay in?
Can I bring my own bike or rent one in Amsterdam?
Are there any unique or quirky experiences I should have while visiting Amsterdam?'''
tts= ElevenLabsText2SpeechTool()
print(tts.name)


eleven_labs_text2speech


In [ ]:
#!pip install --upgrade elevenlabs==0.2.27
speech_file = tts.run(text_to_speak)
speech_file


In [ ]:
#tts.play(speech_file)
Audio(speech_file)

# 5.2 ElevenLabs with Agents

In [ ]:
text_to_speak = '''Top 10 Questions for First-Time Travelers to Amsterdam:
Here are the top 10 questions that a first-time traveler might ask about visiting Amsterdam:
What is the best way to get around Amsterdam?
Is Amsterdam safe for tourists?
What are some must-see attractions in Amsterdam?
Can I drink the tap water in Amsterdam?
Are there any specific dress code or cultural norms I should be aware of?
How much money do I need to budget for food and activities?
Is Amsterdam a good place for solo travelers or couples?
What are some popular neighborhoods or areas to stay in?
Can I bring my own bike or rent one in Amsterdam?
Are there any unique or quirky experiences I should have while visiting Amsterdam?'''
tools = load_tools(["eleven_labs_text2speech"])

agent = initialize_agent(
    tools=tools,
    llm=LLM,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)
audio_file = agent.run(text_to_speak),


NameError: name 'load_tools' is not defined